import all libs for interactive sesion

In [6]:
import pyaudio
# imports
import matplotlib.pyplot as plt
import numpy as np
import wave, sys
import librosa # this is the cooler way to plot the sound
import librosa.display
import pandas as pd
import random
import math
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import scipy
from pydub import AudioSegment
from scipy.io.wavfile import write

C:\Python38-64\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


all file names and globals

In [10]:
filename = "C:\sound\soundfiles\\recorded.wav"
filedir = "soundfiles//"
DATACSVPATH = "data.csv"
FORMAT = pyaudio.paInt16
SAMPLERATE = 44100

Record files, split them, and generate training data

In [30]:
def recordAndSaveToFile(record_seconds = 60,_filename = filename ,channels =1,_sample_rate = SAMPLERATE,_chunk = 1024):
    p = pyaudio.PyAudio()
    stream = p.open( format=FORMAT,
    channels=channels,
    rate=_sample_rate,
    input=True,
    output=True,
    frames_per_buffer=_chunk)

    #record audio
    frames = []
    print("started recording")
    for i in range(int(_sample_rate /_chunk * record_seconds)):
        data = stream.read(_chunk)
        frames.append(data)
    print("stop")
    stream.stop_stream()
    stream.close()
    p.terminate()

    #save to .wav file
    wf = wave.open(filename, "wb")
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(_sample_rate)
    wf.writeframes(b"".join(frames))
    wf.close()

def split_save_files(_filename=filename, _save_dir = filedir, _num_splits = 1):
    temp_aud = AudioSegment.from_file(_filename)
    temp_secs =  int(temp_aud.duration_seconds)

    #number of seconds in each variable
    interval = int(temp_secs / _num_splits)
    print(interval)
    for x in range(0,temp_secs,interval):
        t1 = x * 1000 # mesures in milli
        t2 = (x + interval) * 1000
        clip = temp_aud[t1:t2]
        temp_file_name =_save_dir + str(t1) + "_" + str(t2) + ".wav"
        clip.export(temp_file_name, format="wav")
        #does python go out of range?


def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

class pointGen:
   
    def __init__(self,radius: float,distance_mics = 10):
        self.origin = (0,0)
        self.X_max_value = radius
        self.X_min_value = -radius
        self.Y_max_value = radius
        self.Y_min_value = 0
        self.point_L_mic = (-distance_mics/2,0)
        self.point_R_mic = ( distance_mics/2,0)
        self.SPEEDOFSOUNDCMPS = 34300 #centimeater per second
    def genRandEe(self):
        output = random.uniform(self.X_min_value, self.X_max_value), random.uniform(self.Y_min_value, self.Y_max_value)
        print(output)
        dist_L_mic = math.dist(output,self.point_L_mic)
        dist_R_mic = math.dist(output,self.point_R_mic)
        time_to_L_mic = dist_L_mic / self.SPEEDOFSOUNDCMPS#centimeater per second
        time_to_R_mic = dist_R_mic / self.SPEEDOFSOUNDCMPS#centimeater per second
        return int((time_to_L_mic - time_to_R_mic)*SAMPLERATE),angle_between(output,self.origin)
        #find the distance
        
def gen_file_structure_and_csv(_filedir = filedir,_number_of_shifts_per_instance = 1):
    ch1dir = "ch1//"
    ch2dir = "ch2//"
    d = {'ch1_file': [], 'ch2_file': [], 'angle': [] , 'shift':[]}
    #df = pd.DataFrame(data=d)
    # generate a point
    _pg = pointGen(radius=60000,distance_mics=100)
    for file in os.listdir(_filedir):
        try:
            raw = wave.open(_filedir + file)
            signal = raw.readframes(-1)
            signal = np.frombuffer(signal, dtype ="int16")
            
            for ind in range(_number_of_shifts_per_instance):
                shift,angle = _pg.genRandEe()
                # shift file save copy
                ch2 = np.roll(signal,shift=shift)
                print(file,shift,signal)

                d['ch1_file'].append(str(_filedir+ ch1dir + str(ind) + "_" + file))
                d['ch2_file'].append(str(_filedir+ ch2dir + str(ind) + "_" + file))
                d['angle'].append(angle)
                d['shift'].append(shift)
                print(_filedir + ch1dir + str(ind)+ "_" + file)
                write(str(_filedir+ ch1dir + str(ind) + "_" + file), SAMPLERATE, signal)
                write(str(_filedir+ ch2dir + str(ind) + "_" + file), SAMPLERATE, ch2)
                
        except:
            print(file + "did not shift correctly")
        # split into ch1 dir and ch2 dir
    df = pd.DataFrame(data=d)
    df.to_csv(DATACSVPATH)
    return df
        

In [ ]:
recordAndSaveToFile()

In [27]:
split_save_files(_filename=filename,_save_dir=filedir,_num_splits=59)
df = gen_file_structure_and_csv(_number_of_shifts_per_instance=10)

1
(53107.70965307717, 13498.391135321084)
0_1000.wav 124 [  3   3   2 ... -60 -59 -57]
soundfiles//ch1//0_0_1000.wav
(2659.1251394518185, 52509.838825633444)
0_1000.wav 6 [  3   3   2 ... -60 -59 -57]
soundfiles//ch1//1_0_1000.wav
(48260.63626891206, 38182.63748870825)
0_1000.wav 100 [  3   3   2 ... -60 -59 -57]
soundfiles//ch1//2_0_1000.wav
(52082.62656814212, 16859.55227657612)
0_1000.wav 122 [  3   3   2 ... -60 -59 -57]
soundfiles//ch1//3_0_1000.wav
(-10304.42374528337, 21296.741765843522)
0_1000.wav -55 [  3   3   2 ... -60 -59 -57]
soundfiles//ch1//4_0_1000.wav
(39010.41620123865, 15136.833666367911)
0_1000.wav 119 [  3   3   2 ... -60 -59 -57]
soundfiles//ch1//5_0_1000.wav
(44247.88518069708, 29560.18618402021)
0_1000.wav 106 [  3   3   2 ... -60 -59 -57]
soundfiles//ch1//6_0_1000.wav
(-55549.04639037797, 2576.6648268636104)
0_1000.wav -128 [  3   3   2 ... -60 -59 -57]
soundfiles//ch1//7_0_1000.wav
(53323.74737888333, 26488.633195915227)
0_1000.wav 115 [  3   3   2 ... -60 -59

Training code

In [134]:
def get_dataset(df):
    ch1_file_path_ds = tf.data.Dataset.from_tensor_slices(df.ch1_file)
    ch2_file_path_ds = tf.data.Dataset.from_tensor_slices(df.ch2_file)
    lables = tf.data.Dataset.from_tensor_slices(df.shift)
    return tf.data.Dataset.zip((ch1_file_path_ds,ch2_file_path_ds,lables))
#newds = get_dataset(df)
def get_alt_set():
    ch1_file_path_ds = tf.data.Dataset.from_tensor_slices(df.ch1_file)
    ch2_file_path_ds = tf.data.Dataset.from_tensor_slices(df.ch2_file)
    lables = tf.data.Dataset.from_tensor_slices(df.angle)
    return tf.data.Dataset.zip((ch1_file_path_ds,ch2_file_path_ds,lables))

def load_audio(ch1_file_path,ch2_file_path,label):
    ch1_audio = tf.io.read_file(ch1_file_path)
    ch2_audio = tf.io.read_file(ch2_file_path)
    ch1_audio,samp = tf.audio.decode_wav(ch1_audio,desired_channels=1,desired_samples=SAMPLERATE)
    ch2_audio,samp = tf.audio.decode_wav(ch2_audio,desired_channels=1,desired_samples=SAMPLERATE)
    return tf.concat([ch1_audio,ch2_audio],axis=1),label

def prepare_ds_for_training(ds,_batch):
    ds = ds.shuffle(buffer_size=1)
    ds = ds.map(load_audio,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.repeat()
    ds = ds.batch(_batch)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds




In [135]:
ds = prepare_ds_for_training(get_dataset(pd.read_csv(DATACSVPATH)),_batch=12)


ValueError: Attempt to convert a value (<bound method DataFrame.shift of      Unnamed: 0                         ch1_file  \
0             0    soundfiles//ch1//0_0_1000.wav   
1             1    soundfiles//ch1//1_0_1000.wav   
2             2    soundfiles//ch1//2_0_1000.wav   
3             3    soundfiles//ch1//3_0_1000.wav   
4             4    soundfiles//ch1//4_0_1000.wav   
..          ...                              ...   
595         595  soundfiles//ch1//5_recorded.wav   
596         596  soundfiles//ch1//6_recorded.wav   
597         597  soundfiles//ch1//7_recorded.wav   
598         598  soundfiles//ch1//8_recorded.wav   
599         599  soundfiles//ch1//9_recorded.wav   

                            ch2_file       angle  shift  
0      soundfiles//ch2//0_0_1000.wav   14.260896    124  
1      soundfiles//ch2//1_0_1000.wav   87.100989      6  
2      soundfiles//ch2//2_0_1000.wav   38.350185    100  
3      soundfiles//ch2//3_0_1000.wav   17.937172    122  
4      sound

In [150]:
'''model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv1D(activation='relu',filters=1 , kernel_size=2 ,strides=2),
        #tf.keras.layers.MaxPool1D(pool_size=2,strides=2),
        #tf.keras.layers.Conv1D(activation='relu',filters=1, kernel_size=2 ,strides=2),
        #tf.keras.layers.MaxPool1D(pool_size=2,strides=2),
        #tf.keras.layers.Conv1D(activation='relu',filters=1 , kernel_size=2 ,strides=2),
        #tf.keras.layers.Conv1D(activation='relu',filters=1 , kernel_size=2 ,strides=2),
        #tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(activation='softmax',units=1),# this will be the number of classes # there should be a drop out
    ]
)'''

model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv1D(activation='relu',filters=128 , kernel_size=200 ,strides=2),
        tf.keras.layers.MaxPool1D(pool_size=2,strides=2),
        tf.keras.layers.Conv1D(activation='relu',filters=128, kernel_size=100 ,strides=2),
        tf.keras.layers.MaxPool1D(pool_size=2,strides=2),
        tf.keras.layers.Conv1D(activation='relu',filters=1 , kernel_size=2 ,strides=2),
        tf.keras.layers.Conv1D(activation='relu',filters=1 , kernel_size=2 ,strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(activation='tanh',units=20),# this will be the number of classes # there should be a drop out
    ]
)

learning_rate = .1 
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#outputs are not normalized so from_logits = 0
# this one may be wrong
loss_fn = tf.keras.losses.MeanAbsoluteError(reduction="auto")
metrics =tf.keras.metrics.MeanAbsoluteError()
model.compile(optimizer=optimizer,loss=loss_fn,metrics=metrics)



In [151]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [152]:
#yeet
model.fit(ds,epochs=10,steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 326s 3s/step - loss: 87.7291 - mean_absolute_error: 87.7291
Epoch 2/10
 42/100 [===========>..................] - ETA: 12:46 - loss: 87.4380 - mean_absolute_error: 87.4380